In [2]:
import pandas as pd
import numpy as np
import os
import requests

## Sprawdzanie po trans_formed

In [136]:
crash_fact = pd.read_pickle('./data/tmp/transformed/fact_crash.pkl')
dim_vehicles = pd.read_pickle('./data/tmp/transformed/dim_vehicle.pkl')
dim_crash_info = pd.read_pickle('./data/tmp/transformed/dim_crash_info.pkl')
fact_weather = pd.read_pickle('./data/tmp/transformed/fact_weather.pkl')
dim_people = pd.read_pickle('./data/tmp/transformed/dim_person.pkl')

In [76]:
crash_fact.head(5)

,CRASH_RECORD_ID,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,date_id,FACT_CRASH_KEY
0,6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...,2,1,0,0,1,0,1,0,20230818,18298157505609667498
1,5f54a59fcb087b12ae5b1acff96a3caf4f2d37e79f8db4...,4,0,0,0,0,0,1,0,20230729,6291736621306781135
2,61fcb8c1eb522a6469b460e2134df3d15f82e81fd93e9c...,2,1,0,0,1,0,1,0,20230818,10536150198451459749
3,004cd14d0303a9163aad69a2d7f341b7da2a8572b2ab33...,2,1,1,0,0,0,1,0,20191126,1153394593627935104
4,a1d5f0ea90897745365a4cbb06cc60329a120d89753fac...,1,0,0,0,0,0,1,0,20230818,6365068776030650289


In [77]:
dim_crash_info.head(5)

,CRASH_INFO_KEY,CRASH_RECORD_ID,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,REPORT_TYPE,CRASH_TYPE,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,MOST_SEVERE_INJURY
0,1178550305476037711,6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...,OTHER,FUNCTIONING PROPERLY,REAR END,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",FOLLOWING TOO CLOSELY,DISTRACTION - FROM INSIDE VEHICLE,NONINCAPACITATING INJURY
1,14321821260077918184,5f54a59fcb087b12ae5b1acff96a3caf4f2d37e79f8db4...,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,"OVER $1,500",FAILING TO REDUCE SPEED TO AVOID CRASH,"OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELE...",NO INDICATION OF INJURY
2,14412299524608918156,61fcb8c1eb522a6469b460e2134df3d15f82e81fd93e9c...,NO CONTROLS,NO CONTROLS,PEDALCYCLIST,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,"$501 - $1,500",FAILING TO REDUCE SPEED TO AVOID CRASH,UNABLE TO DETERMINE,NONINCAPACITATING INJURY
3,7017221841570418401,004cd14d0303a9163aad69a2d7f341b7da2a8572b2ab33...,NO CONTROLS,NO CONTROLS,PEDESTRIAN,ONE-WAY,CURVE ON GRADE,DRY,NO DEFECTS,ON SCENE,INJURY AND / OR TOW DUE TO CRASH,"OVER $1,500",UNABLE TO DETERMINE,NOT APPLICABLE,FATAL
4,14603271235315365445,a1d5f0ea90897745365a4cbb06cc60329a120d89753fac...,NO CONTROLS,NO CONTROLS,FIXED OBJECT,OTHER,STRAIGHT AND LEVEL,DRY,NO DEFECTS,ON SCENE,NO INJURY / DRIVE AWAY,"OVER $1,500",FOLLOWING TOO CLOSELY,DRIVING SKILLS/KNOWLEDGE/EXPERIENCE,NO INDICATION OF INJURY


In [137]:
dim_people.head(5)

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,VEHICLE_ID,CRASH_DATE,SEX,AGE,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,EJECTION,INJURY_CLASSIFICATION,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,BAC_RESULT,BAC_RESULT VALUE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834816,09/28/2019 03:30:00 AM,M,25,NONE PRESENT,UNKNOWN,NONE,NO INDICATION OF INJURY,UNKNOWN,UNKNOWN,UNKNOWN,TEST NOT OFFERED,-999.0
1,O663866,DRIVER,f09cdb96b5c9d25a1f82573805eb40067ac1555bc35cda...,632057,06/06/2019 12:40:00 PM,F,-1,SAFETY BELT USED,NOT APPLICABLE,NONE,NO INDICATION OF INJURY,NONE,NOT OBSCURED,NORMAL,TEST NOT OFFERED,-999.0
2,O848861,DRIVER,d08b8b938cc6777e1d3f5a5c299649829d812336fb6679...,805584,11/09/2019 03:20:00 PM,M,56,UNKNOWN,UNKNOWN,NONE,NO INDICATION OF INJURY,IMPROPER LANE CHANGE,UNKNOWN,UNKNOWN,TEST NOT OFFERED,-999.0
3,O850909,DRIVER,cd68e3218ffe768cfc1a5e0c75fd3a54365ef1d53e67c5...,807488,10/20/2019 05:20:00 PM,M,46,UNKNOWN,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,IMPROPER TURN,UNKNOWN,NORMAL,TEST NOT OFFERED,-999.0
4,O851790,DRIVER,9a87e7f29da8636e57cbd77bcb69a3cc352ef4976b02bb...,808333,07/30/2019 10:30:00 AM,M,60,UNKNOWN,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,IMPROPER BACKING,NOT OBSCURED,NORMAL,TEST NOT OFFERED,-999.0
